In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import asyncio
import aiohttp
import nest_asyncio
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from loguru import logger
from datetime import datetime
import glob
from matplotlib import pyplot as plt
from collections import Counter
import time
from fake_useragent import UserAgent
from collections import Counter
import urllib

nest_asyncio.apply()


## Спарсим рамблер

In [ ]:
signs = ['aries', 'gemini', 'taurus', 'cancer', 'leo', 'virgo', 'libra', 'scorpio', 'sagittarius', 'capricorn',
         'aquarius', 'pisces']
data = pd.read_csv('data.csv', names=['date', 'sign', 'text'])

for i in range(2004, 2021):
    for j in range(1, 13):
        for k in range(1, 32):
            for sign in signs:
                try:
                    year = i
                    month = j
                    day = k
                    if month < 10:
                        month = '0' + str(month)
                    if day < 10:
                        day = '0' + str(day)
                    date = '%s-%s-%s' % (year, month, day)
                    url = 'https://horoscopes.rambler.ru/%s/%s/?updated' % (sign, date)  # url для второй страницы
                    print(url)
                    r = requests.get(url)
                    response = r.text.encode('utf-8')

                    soup = BeautifulSoup(response, features="lxml")
                    text = soup.find('div', {'class': '_1dQ3'}).text
                    data = data.append({'date': date, 'sign': sign, 'text': text}, ignore_index=True)
                    data.to_csv('data.csv', encoding='utf-8')
                    print('saved date=', date, ' sign=', sign)
                except Exception:
                    continue

## Сайт Ростова-на-Дону

https://www.1rnd.ru/

In [ ]:
news = []
url = 'https://www.1rnd.ru/news/3089599'
print(url)
r = requests.get(url)
response = r.text
soup = BeautifulSoup(response, 'html.parser')

news_text = soup.findAll('p', {'style': "text-align:justify;"})
date_text = soup.findAll('div', {'class': "article-info__time"})
header_text = soup.findAll('div', {'class': 'title-container inner-title'})

if news_text:
    print('success')
    text_line = ''
    for text in news_text:
        text_line = text_line + text.get_text()
    news.append([header_text[0].get_text(), date_text[0].get_text().split(), text_line])

In [ ]:
news = pd.DataFrame({'header': [0], 'date': [0], 'news': [0]})

In [ ]:
start1 = 7135
start2 = 300000
start3 = 600000
start4 = 900000
start5 = 1200000
start6 = 1500000
start7 = 1800000
start8 = 2100000
start9 = 2400000
start10 = 2700000

end1 = 299999
end2 = 599999
end3 = 899999
end4 = 1199999
end5 = 1499999
end6 = 1799999
end7 = 2099999
end8 = 2399999
end9 = 2699999
end10 = 2999999


async def get_news(start, end):
    global news
    for i in range(start, end):
        url = 'https://www.1rnd.ru/news/{0}'.format(str(i))
        print(url)

        async with aiohttp.ClientSession() as session:
            async with session.get(url) as response:

                response = await response.text()
                soup = BeautifulSoup(response, 'html.parser')

                news_text = soup.findAll('p', {'style': "text-align:justify;"})
                date_text = soup.findAll('div', {'class': "article-info__time"})
                header_text = soup.findAll('div', {'class': 'title-container inner-title'})

                if news_text:
                    print('success')
                    text_line = ''
                    for text in news_text:
                        text_line = text_line + text.get_text()

                    date = date_text[0].get_text()
                    header = header_text[0].get_text()

                    news = news.append({'header': header, 'date': date, 'news': text_line}, ignore_index=True)
                    news.to_csv('news.csv', encoding='utf-8')


futures = [get_news(start1, end1),
           get_news(start2, end2),
           get_news(start3, end3),
           get_news(start4, end4),
           get_news(start5, end5),
           get_news(start6, end6),
           get_news(start7, end7),
           get_news(start8, end8),
           get_news(start9, end9),
           get_news(start10, end10)]

loop = asyncio.get_event_loop()
loop.run_until_complete(asyncio.wait(futures))
loop.close()

Товарная единица - ивент

Нужно определить - 
 - [x] категория
 - [x] стоимость 
 - [x] геопозиция
 - [x] дата
 - [x] длительность
 - [x] количество людей
 - [ ] список услуг (недоступно)
 - [x] кто проводит
 - [x] сайт

# BookYogaRetreats

In [ ]:
page = 1
base_url = 'www.bookyogaretreats.com'
url = f'https://{base_url}/?page={page}'
print(url)
r = requests.get(url)
response = r.text.encode('utf-8')

soup = BeautifulSoup(response)
text = soup.findAll('a', {'class': 'js-showcard-link'}, href=True)
card_urls = []
for t in text:
    card_urls.append(t['href'])
# data=data.append({'date': date, 'sign': sign, 'text': text}, ignore_index=True)
# data.to_csv('data.csv', encoding = 'utf-8')
# print('saved date=', date, ' sign=', sign)

In [ ]:
card_urls

In [ ]:
page = 1
base_url = 'www.bookyogaretreats.com'
url = f'https://{base_url}{card_urls[10]}'
# url='https://www.bookyogaretreats.com/yoga-weeks/6-day-urban-yoga-retreat-in-barcelona-catalonia'
print(url)
r = requests.get(url)
response = r.text.encode('utf-8')

soup = BeautifulSoup(response, 'html.parser')
text = soup.findAll('a', {'class': 'js-showcard-link'})

In [ ]:
soup

In [ ]:
soup.findAll('div', {'class': 'pre-main-content__location'})[0].text

In [ ]:
soup.findAll('div', {'class': 'listing-title'})[0].text

# Completed


In [ ]:
duration = soup.findAll('div', {
    'class': 'subtitle-small listing-query-box__duration js-sc3-inquiry-modal-duration js-insert-duration'})[0].text
duration = duration.replace('\n', '').split(' ')[0]
duration

In [ ]:
title = soup.findAll('h1', {'class': 'listing-title__title title listing-title-new'})[0].text
title

In [ ]:
arrival = soup.findAll('span', {'class': 'js-sc1-inquiry-modal-arrival'})
if len(arrival) != 0:
    arrival = arrival[0].text
    arrival = arrival.replace(',', '').split(' ')
    arrival[1] = arrival[1][:3]
    arrival = ' '.join(arrival[1:])
    arrival = datetime.strptime(arrival, '%b %d %Y').date()
else:
    arrival = None

print(arrival)

In [ ]:
departure = soup.findAll('span', {'class': 'js-sc1-inquiry-modal-departure'})
if len(departure) != 0:
    departure = departure[0].text
    departure = departure.replace(',', '').split(' ')
    departure[1] = departure[1][:3]
    departure = ' '.join(departure[1:])
    departure = datetime.strptime(departure, '%b %d %Y').date()
else:
    departure = None

print(departure)

In [ ]:
datetime.strptime('May 17 2022', '%b %d %Y').date().isoformat()

In [ ]:
organizer = soup.findAll('div', {'class': 'listing-organizer-card__content'})[0].text[20:].split('\n')[0]
organizer

In [ ]:
arr = soup.findAll('div', {'class': 'listing-overview__notes'})[0].text.split('\n')
arr = [a for a in arr if a != '']
duration = arr[0].split(' ')[0]
duration

In [ ]:
location = soup.findAll('div', {'class': 'listing-title__location listing-location-new'})
if len(location[0]) != 0:
    location = location[0].text
else:
    location = None

print(location)

In [ ]:
arr = soup.findAll('div', {'class': 'listing-overview__notes'})[0].text.split('\n')
arr = [a for a in arr if a != '']
group_size = [a for a in arr if 'Group' in a]

if len(group_size) != 0:
    if len(group_size[0].split(' ')) == 6:
        group_size = group_size[0].split(' ')[4]
    elif len(group_size[0].split(' ')) == 4:
        group_size = group_size[0].split(' ')[2]
else:
    group_size = None

print(arr)
print(group_size)

In [ ]:
driver = webdriver.Firefox(executable_path=r"D:\загрузки\geckodriver-v0.30.0-win64\geckodriver.exe")

In [ ]:
driver.get(url)
driver.find_elements_by_class_name('price')[0].text.split(' ')[1]

In [ ]:
#main page

main_page_url = 'https://www.bookyogaretreats.com'
r = requests.get(main_page_url)
response = r.text.encode('utf-8')

soup = BeautifulSoup(response)

categories_raw = soup.findAll('li', {'class': 'fakeCheck'})
categories_raw = [' '.join(category.text.replace('\n', '').split(' ')[:-1]) for category in categories_raw][12:-4]
categories = []

for i, category in enumerate(categories_raw):
    if 'level' in category.lower():
        categories.append(category.split(' ')[1])
    else:
        categories.append(category)

# categories

In [ ]:
soup.findAll('div', {'id': 'error-page'})

# Pipeline

In [ ]:
'https://www.bookyogaretreats.com/sr?c=budget-retreats&page=1'

In [ ]:
data = pd.DataFrame({'title': [0], 'duration': [0], 'arrival': [0], 'departure': [0], 'organizer': [0], 'location': [0],
                     'group_size': [0], 'price': [0]})
# data_categories={}

In [ ]:
main_page_url = 'https://www.bookyogaretreats.com'

ua = UserAgent(verify_ssl=False)
headers = {'User-Agent': ua.random}

r = requests.get(main_page_url, headers=headers)
response = r.text.encode('utf-8')
soup = BeautifulSoup(response)

categories_raw = soup.findAll('li', {'class': 'fakeCheck'})
categories_raw = [' '.join(category.text.replace('\n', '').split(' ')[:-1]) for category in categories_raw][12:-4]
categories = []

driver = webdriver.Firefox(executable_path=r"D:\загрузки\geckodriver-v0.30.0-win64\geckodriver.exe")

for i, category in enumerate(categories_raw):
    if 'level' in category.lower():
        categories.append(category.split(' ')[1])
    else:
        categories.append(category)

# c_u=category_page_url=main_page_url+f'/sr?c={cat}&page={page}'

for page in range(428, 429):
    #     cat='-'.join(category.split(' ')).lower()
    cat = 'cat '
    #     stop=False

    #     data_categories[cat]=[]
    #     while stop is False:

    page_url = main_page_url + f'/?page={page}'
    print('NEW Page ', page_url, '\n----------------------')

    # страница каталога 
    r = requests.get(page_url, headers=headers)
    response = r.text.encode('utf-8')
    soup = BeautifulSoup(response)

    # если такой страницы нет, т.е. каталог закончился
    error = soup.findAll('div', {'id': 'error-page'})

    if len(error) > 0:
        stop = True
        print(f'####Category page {page} ERROR####')
        break

    text = soup.findAll('a', {'class': 'js-showcard-link'}, href=True)
    card_urls = []
    for t in text:
        card_urls.append(t['href'])

    for card_url in card_urls:

        url = f'{main_page_url}{card_url}'
        print(f'category {cat}, url {url}')
        r = requests.get(url, headers=headers)
        response = r.text.encode('utf-8')
        soup = BeautifulSoup(response, 'html.parser')

        error = soup.findAll('div', {'id': 'error-page'})
        if len(error) > 0:
            continue

        arr = soup.findAll('div', {'class': 'listing-overview__notes'})[0].text.split('\n')
        arr = [a for a in arr if a != '']
        duration = arr[0].split(' ')[0]

        title = soup.findAll('h1', {'class': 'listing-title__title title listing-title-new'})[0].text
        #             data_categories[cat].append(title)

        arrival = soup.findAll('span', {'class': 'js-sc1-inquiry-modal-arrival'})
        if len(arrival) != 0:
            arrival = arrival[0].text
            arrival = arrival.replace(',', '').split(' ')
            arrival[1] = arrival[1][:3]
            arrival = ' '.join(arrival[1:])
            arrival = datetime.strptime(arrival, '%b %d %Y').date()
        else:
            arrival = None

        departure = soup.findAll('span', {'class': 'js-sc1-inquiry-modal-departure'})
        if len(departure) != 0:
            departure = departure[0].text
            departure = departure.replace(',', '').split(' ')
            departure[1] = departure[1][:3]
            departure = ' '.join(departure[1:])
            departure = datetime.strptime(departure, '%b %d %Y').date()
        else:
            departure = None

        organizer = soup.findAll('div', {'class': 'listing-organizer-card__content'})[0].text[20:].split('\n')[0]

        location = soup.findAll('div', {'class': 'listing-title__location listing-location-new'})
        if len(location[0]) != 0:
            location = location[0].text
        else:
            location = None

        arr = soup.findAll('div', {'class': 'listing-overview__notes'})[0].text.split('\n')
        arr = [a for a in arr if a != '']
        group_size = [a for a in arr if 'Group' in a]

        if len(group_size) != 0:
            if len(group_size[0].split(' ')) == 6:
                group_size = group_size[0].split(' ')[4]
            elif len(group_size[0].split(' ')) == 4:
                group_size = group_size[0].split(' ')[2]
        else:
            group_size = None

        driver.get(url)

        price_list = driver.find_elements_by_class_name('price')
        if len(price_list) > 0:
            if len(price_list[0].text.split(' ')) < 2:
                price = None
            else:
                price = price_list[0].text.split(' ')[1]
        else:
            price = None

        data = data.append({'title': [title], 'duration': [duration], 'arrival': [arrival], 'departure': [departure],
                            'organizer': [organizer], 'location': [location], 'group_size': [group_size],
                            'price': [price]}, ignore_index=True)
    data.to_csv('data.csv', encoding='utf-8')
#         np.save('data_categories.npy',data_categories)

#         page+=1

### Только страны

In [ ]:
data_countries = pd.DataFrame({'title': [0], 'country': [0]})
# data_categories={}

In [ ]:
main_page_url = 'https://www.bookyogaretreats.com'

ua = UserAgent(verify_ssl=False)
headers = {'User-Agent': ua.random}

for page in range(0, 429):

    cat = 'cat '

    page_url = main_page_url + f'/?page={page}'
    print('NEW Page ', page_url, '\n----------------------')

    # страница каталога 
    r = requests.get(page_url, headers=headers)
    response = r.text.encode('utf-8')
    soup = BeautifulSoup(response)

    countries = [country.text for country in soup.findAll('div', {'class': 'pre-main-content__location'})]
    titles = [title.text for title in soup.findAll('div', {'class': 'listing-title'})]

    for i, title in enumerate(titles):
        data_countries = data_countries.append({'title': title[1:-1], 'country': countries[i][1:-1]}, ignore_index=True)
    data_countries.to_csv('data_countries.csv', encoding='utf-8')

In [ ]:
cleaned_countries = data_countries.drop_duplicates()
cleaned_countries = cleaned_countries.drop(cleaned_countries.index[[0]])

In [ ]:
cleaned_countries.head()
cleaned_countries.to_csv('cleaned_countries.csv', encoding='utf-8')

In [ ]:
for line in cleaned_countries.values:
    print(line[1])

In [ ]:
cat = '-'.join(categories[0].split(' '))
category_page_url = main_page_url + f'/sr?c={cat}'

In [ ]:
df_paths = glob.glob('byr/*')
dfs = [pd.read_csv(path) for path in df_paths]
data = pd.concat(dfs)
del data['Unnamed: 0']
data_df = data.drop_duplicates()
data = data_df.values[1:]

In [ ]:
df_paths = glob.glob('byr/*')
dfs = [pd.read_csv(path) for path in df_paths]
data = pd.concat(dfs)
del data['Unnamed: 0']
data_df = data.drop_duplicates()
data = data_df.values[1:]

for i, line in enumerate(data):

    title, duration, arrival, departure, organizer, location, group_size, price = line
    title = title[2:-2]
    duration = float(duration[2:-2])

    if len(arrival) > 6:
        arrival = arrival[15:-2].replace(',', '').replace(' ', '-')
    else:
        arrival = None

    if len(departure) > 6:
        departure = departure[15:-2].replace(',', '').replace(' ', '-')
    else:
        departure = None

    organizer = organizer[2:-2]
    location = location[2:-2]

    if 'None' in group_size:
        group_size = None
    elif '-' in group_size:
        group_size = group_size[2:-2].split('-')[1]
        group_size = float(group_size)
    else:
        group_size = float(group_size[2:-2])

    if 'None' in price:
        price = None
    else:
        price = price[2:-2].replace(',', '')
        price = float(price)

    data[i] = [title, duration, arrival, departure, organizer, location, group_size, price]



In [ ]:
cleaned_data = pd.DataFrame(
    {'title': data[:, 0], 'duration': data[:, 1], 'arrival': data[:, 2], 'departure': data[:, 3],
     'organizer': data[:, 4], 'location': data[:, 5], 'group_size': data[:, 6], 'price': data[:, 7]})
cleaned_data.to_csv('cleaned_data.csv', encoding='utf-8')

In [ ]:
# количество ретритов по годам и без даты

step = 1
year = '2022'
for line in cleaned_data.values:
    # if line[2] is None:
    if line[2][:4]==year:
        step += 1
step

In [ ]:
cleaned_data['price'].astype(float).describe()

In [ ]:
# объем рынка

total = 0
for line in cleaned_data.values:
    price = line[-1]
    group_size = line[-2]
    if price is not None and group_size is not None:
        total += price * group_size
total

In [ ]:
# количество уникальных ретитов по датам
month_list=['01','02','03','04','05','06','07','08','09','10','11','12']

for month_comp in month_list:
    titles = []
    for line in cleaned_data.values:
        if line[2] is not None:
            year, month, day = line[2].split('-')
            title = line[0]
            if str(month_comp) == month:
                titles.append(title)

    month_df = pd.DataFrame({'titles': titles}).drop_duplicates()

    print(len(month_df))

In [ ]:
# количество уникальных ретитов по годам
year_list=['2022','2023','2024']

for year_comp in year_list:
    titles = []
    for line in cleaned_data.values:
        if line[2] is not None:
            year, month, day = line[2].split('-')
            title = line[0]
            # if str(month_comp) == month:
            if year_comp == year:
                titles.append(title)

    month_df = pd.DataFrame({'titles': titles}).drop_duplicates()

    print(len(month_df))

Количество ретритов на сайте (шт) - 4650

Количество ретритов с привязкой к дате (шт): 2022г - 1437, 2023г - 36, 2024г - 2

Количество ретритов без привязки к дате (шт): 3181

Количество уникальных ретритов с привязкой по дате по месяцам (шт): январь - 13, февраль - 64, март - 238, апрель - 246, май - 212, июнь - 183, июль - 143, август - 101, сентябрь - 99, ноябрь - 56, декабрь - 28 



Цена (евро): среднее - 1243, медиана - 900,  дисперсия - 1190, миниум - 18, максимум - 27300 

Длительность (дни): среднее - 10, медиана - 6, дисперсия - 16, минимум - 1, максимум 365

Количество человек (шт): среднее - 13.5, медиана - 12, дисперсия - 12, минимум - 1, максимум 500

Сумма стоимости ретритов х количество участников (млн евро) - 56.25

In [ ]:
cleaned_data = pd.read_csv('bookyogaretreat.csv')

In [ ]:
contries = []
for line in cleaned_data.values:
    cnt = line[6]
    contries.append(cnt)
contries.sort()
#144
contries = contries[0:]
contries

In [ ]:
values = contries
counts = Counter(values)
contries_x = []
contries_y = []
for k, count in counts.most_common():
    contries_x.append(k)
    contries_y.append(count)
    print(k, count * 'x')


In [ ]:
N = 30
plt.figure(figsize=(30, 15))
plt.plot(contries_x[:N], contries_y[:N])
plt.yticks(np.arange(0, 450, 25))
plt.grid(True)
plt.legend(['количество ретритов за все время по странам'], fontsize=20)
# plt.savefig('ретриты по странам.png')
plt.show()

In [ ]:
# for line in contries_y:
for line in contries_x:
    print(line)


In [ ]:
dpi = 150
fig = plt.figure(dpi=dpi, figsize=(15, 30))

ax = plt.axes()
ax.xaxis.grid(True)

N = 60

plt.barh(contries_x[:N], contries_y[:N], label='количество ретритов за все время по странам')
plt.xticks(np.arange(0, 450, 25))
plt.legend(loc='upper right')
plt.savefig('ретриты по странам.png')
plt.show()

# Book Retreats

Товарная единица - ивент

Нужно определить -
- [ ] категория
- [x] стоимость
- [x] геопозиция
- [x] дата
- [x] длительность
- [ ] количество людей (недоступно)
- [ ] список услуг (недоступно)
- [x] кто проводит (недоступно)
- [x] сайт

In [ ]:
page = 1
# base_url='https://bookretreats.com/'
base_url = 'https://bookretreats.com'
# url = f'{base_url}/?page={page}'
url = 'https://bookretreats.com/s/other-retreats?pageNumber=1'
# url='https://bookretreats.com/7-day-detox-recharge-retreat-in-nature-sri-lanka'
print(url)
r = requests.get(url)
response = r.text.encode('utf-8')

soup = BeautifulSoup(response, 'html.parser')

In [ ]:
titles = soup.findAll('h3', {'class': 'text-xl text-gray-600 dark:text-gray-200 font-semibold mb-2'})
titles = [title.text for title in titles]
# soup.findAll('div', {'class': 'flex flex-col lg:justify-between h-full px-2 sm:p-0'})[0].text
text = soup.findAll('a', {'class': 'col-span-12 sm:col-span-7'}, href=True)
card_urls = []
for t in text:
    card_urls.append(t['href'])

card_urls = [base_url + card for card in card_urls]
card_urls

In [ ]:
page = 1
# base_url='https://bookretreats.com/'
base_url = 'https://bookretreats.com'

url = 'https://bookretreats.com/14-day-beginners-yoga-retreat-at-ayuryoga-eco-ashram-india'
print(url)
r = requests.get(url)
response = r.text.encode('utf-8')

soup = BeautifulSoup(response, 'html.parser')
# soup

In [ ]:
soup.findAll('a', {'class': 'Button_textLink__2Ju0p Button_grayColor__2nH6m Button_mdSize__25L9h'})[1].text

In [ ]:
title = soup.findAll('div', {'class': 'col-span-4 sm:col-span-3 flex items-center'})[0].text
print('title ', title)
duration = soup.findAll('div', {'class': 'mb-4 text-base text-gray-500 dark:text-gray-300'})[0].text.split(' ')[0]
print('duration ', duration)
location = soup.findAll('a', {'class': 'Button_textLink__2Ju0p Button_grayColor__2nH6m Button_mdSize__25L9h'})[0].text
print('location ', location)
country = location.split(' ')[-1]
print('county ', country)
deposit = soup.findAll('div', {'class': 'tabular-nums text-xl text-gray-500 dark:text-gray-200 font-semibold'})[0].text[
          1:].replace(',', '')
print('deposit ', deposit)
price = soup.findAll('div', {'class': 'tabular-nums text-2xl text-gray-500 dark:text-gray-200 font-extrabold'})[0].text[
        1:].replace(',', '')
print('price ', price)
category = soup.findAll('a', {'class': 'inline text-sm leading-7 text-gray-500 dark:text-gray-400'})[1].text
print('category ', category)

In [ ]:
driver = webdriver.Firefox(executable_path=r"D:\загрузки\geckodriver-v0.30.0-win64\geckodriver.exe")

In [ ]:
url = 'https://bookretreats.com/10-day-exploding-love-camp-in-akumal-riviera-maya-mexico?searchId=7518398'
url = 'https://bookretreats.com/6-day-yogi-adventure-meditation-in-sintra-portugal?searchId=7792765'

driver.get(url)
print(url)

In [ ]:
# soup

In [ ]:
elem = driver.find_element_by_xpath(
    '//div[@class="Sidebar_sticky__5WOBf sticky top-24 mb-10"]//span[contains(text(),"Select Dates")]')
# elem.click()
# driver.execute_script("arguments[0].click();", elem)

ActionChains(driver).move_to_element(elem).click().perform()
dates = driver.find_elements_by_xpath('//div[@class="lg:max-h-72 overflow-scroll hide-scrollbars"]')
print(dates)

In [ ]:
x = driver.find_elements_by_xpath('//div[@class="lg:max-h-72 overflow-scroll hide-scrollbars"]')
x[0].text

In [ ]:
date =
x[0].text.replace('2022', '2022;').replace('2023', '2023;').replace('2024', '2024;').replace('2025', '2025;').split(
    ';')[:-1][1]

raw_date = date.split(',')
year = raw_date[-1].replace(' ', '')
arrival, departure = raw_date[0].split('-')

arrival_month = arrival.split(' ')[0]
departure_month = departure.split(' ')[-2]

if departure_month == '':
    departure = arrival_month + departure
else:
    departure = departure[1:]

arrival = arrival + year
departure = departure + ' ' + year

arrival = datetime.strptime(arrival, '%b %d %Y').date().isoformat()
departure = datetime.strptime(departure, '%b %d %Y').date().isoformat()

print('arrival ', arrival)
print('departure ', departure)

## Pipeline

In [ ]:
data = pd.DataFrame({'title': [0], 'duration': [0], 'arrival': [0], 'departure': [0],
                     'organizer': [0], 'location': [0], 'country': [0], 'group_size': [0], 'category': [0],
                     'price': [0], 'deposit': [0]})

In [ ]:
data=pd.read_csv('bookretreats.csv')
del data['Unnamed: 0']

In [ ]:
start = 1
stop = 1000

base_url = 'https://bookretreats.com'
page_category = 'wellness-retreats'
driver = webdriver.Firefox(executable_path=r"D:\загрузки\geckodriver-v0.30.0-win64\geckodriver.exe")
driver.maximize_window()

for page in range(start, stop):
    url = f'https://bookretreats.com/s/{page_category}?pageNumber={page}'

    print('-----------------')
    print(f'i={page} new card page ', url)

    r = requests.get(url)
    response = r.text.encode('utf-8')

    soup = BeautifulSoup(response, 'html.parser')
    titles = soup.findAll('h3', {'class': 'text-xl text-gray-600 dark:text-gray-200 font-semibold mb-2'})
    titles = [title.text for title in titles]

    text = soup.findAll('a', {'class': 'col-span-12 sm:col-span-7'}, href=True)
    card_urls = []
    for t in text:
        card_urls.append(t['href'])

    card_urls = [base_url + card for card in card_urls]

    for card_url in card_urls:
        print(card_url)
        r = requests.get(card_url)
        response = r.text.encode('utf-8')
        soup = BeautifulSoup(response, 'html.parser')
        errors = []
        #
        try:
            title = soup.findAll('div', {'class': 'col-span-4 sm:col-span-3 flex items-center'})[0].text
        except Exception:
            title = None
            errors.append('title')

        try:
            duration = soup.findAll('div', {'class': 'mb-4 text-base text-gray-500 dark:text-gray-300'})[0].text.split(' ')[
                0]
        except Exception:
            duration = None
            errors.append('duration')

        try:
            location = soup.findAll('a', {'class': 'Button_textLink__2Ju0p Button_grayColor__2nH6m Button_mdSize__25L9h'})[
                0].text
            if 'reviews' in location:
                location = soup.findAll('a', {'class': 'Button_textLink__2Ju0p Button_grayColor__2nH6m Button_mdSize__25L9h'})[
                    1].text
        except Exception:
            location = None
            errors.append('location')

        try:
            country = location.split(' ')[-1]
        except Exception:
            country = None
            errors.append('country')

        try:
            deposit = soup.findAll('div', {'class': 'tabular-nums text-xl text-gray-500 dark:text-gray-200 font-semibold'})[
                0].text[1:].replace(',', '')
        except Exception:
            deposit = None
            errors.append('deposit')

        try:
            price = soup.findAll('div', {'class': 'tabular-nums text-2xl text-gray-500 dark:text-gray-200 font-extrabold'})[
                0].text[1:].replace(',', '')
        except Exception:
            price = None
            errors.append('price')
        try:
            category = soup.findAll('a', {'class': 'inline text-sm leading-7 text-gray-500 dark:text-gray-400'})[1].text
        except Exception:
            category = None
            errors.append('category')

        driver.get(card_url)

        elem = None
        try:
            elem = driver.find_element_by_xpath(
                '//div[@class="Sidebar_sticky__5WOBf sticky top-24 mb-10"]//span[contains(text(),"Select Dates")]')
        except Exception:
            print('no button')
            elem = False

        if elem:

            ActionChains(driver).move_to_element(elem).click().perform()

            dates = driver.find_elements_by_xpath('//div[@class="lg:max-h-72 overflow-scroll hide-scrollbars"]')
            if len(dates) > 0:
                dates = dates[0].text.replace('2022', '2022;').replace('2023', '2023;').replace('2024',
                                                                                                '2024;').replace('2025',
                                                                                                                 '2025;').split(
                    ';')[:-1]
                for date in dates:
                    year = None
                    arrival = None
                    departure = None
                    try:
                        raw_date = date.split(',')
                        year = raw_date[-1].replace(' ', '')
                        arrival, departure = raw_date[0].split('-')

                        arrival_month = arrival.split(' ')[0]
                        departure_month = departure.split(' ')[-2]

                        if departure_month == '':
                            departure = arrival_month + departure
                        else:
                            departure = departure[1:]

                        arrival = arrival + year
                        departure = departure + ' ' + year

                        arrival = datetime.strptime(arrival, '%b %d %Y').date().isoformat()
                        departure = datetime.strptime(departure, '%b %d %Y').date().isoformat()
                    except Exception:
                        continue

                    data = data.append(
                        {'title': [title], 'duration': [duration], 'arrival': [arrival], 'departure': [departure],
                         'organizer': [None], 'location': [location], 'country': [country], 'group_size': [None],
                         'category': [category],
                         'price': [price], 'deposit': [deposit]}, ignore_index=True)
                data.to_csv('bookretreats.csv', encoding='utf-8')

        # driver.refresh()

In [ ]:
data.info()

In [ ]:
data=pd.read_csv('bookretreats.csv')
del data['Unnamed: 0']

In [ ]:
data_np = data.values[1:]

for i, line in enumerate(data_np):
    title, duration, arrival, departure, organizer, location, country, group_size, category, price, deposit = line
    title = title[2:-2]
    duration = int(duration[2:-2])
    arrival = arrival[2:-2]
    departure = departure[2:-2]
    organizer = None
    location = location[2:-2]
    country = country[2:-2]
    group_size = None
    category = category[2:-2]
    price = int(price[2:-2])
    deposit = int(deposit[2:-2])

    data_np[i] = [title, duration, arrival, departure, organizer, location, country, group_size, category, price,
                  deposit]

cleaned_data = pd.DataFrame(
    {'title': data_np[:, 0], 'duration': data_np[:, 1], 'arrival': data_np[:, 2], 'departure': data_np[:, 3],
     'organizer': data_np[:, 4], 'location': data_np[:, 5], 'country': data_np[:, 6], 'group_size': data_np[:, 7],
     'category': data_np[:, 8],
     'price': data_np[:, 9], 'deposit': data_np[:, 10]})

cleaned_data = cleaned_data.drop_duplicates()

In [ ]:
cleaned_data['duration'].astype(float).describe()

Количество полученных ретритов  (шт) - 3235

Количество ретритов с привязкой к дате (шт): 2022г - 2370, 2023г - 457, 2024г - 141

Количество ретритов без привязки к дате (шт): 38

Количество уникальных ретритов с привязкой по дате по месяцам (шт): январь - 42, февраль - 49, март - 103, апрель - 182, май - 215, июнь - 186, июль - 166, август - 137, сентябрь - 180, октябрь - 159,  ноябрь - 113, декабрь - 92



Цена (евро): среднее - 1061, медиана - 792,  дисперсия - 1014, миниум - 108, максимум - 8115

Длительность (дни): среднее - 8.3, медиана - 7, дисперсия - 6.4, минимум - 2, максимум 29

Количество человек (шт): нет информации

Сумма стоимости ретритов х количество участников (млн евро) - нет информации

## Retreet centres

In [ ]:
url = 'https://bookretreats.com/center/chteau-de-rhodes'
# url = 'https://bookretreats.com/center/42-acres'
print(url)
r = requests.get(url)
response = r.text
soup = BeautifulSoup(response, 'html.parser')
# soup

In [ ]:
imgURL = "http://site.meishij.net/r/58/25/3568808/a3568808_142682562777944.jpg"

urllib.request.urlretrieve(imgURL, "local-filename.jpg")

In [ ]:
# страна и ссылка на tripadvisor
name=soup.findAll('h1',{'class':'text-2xl sm:text-3xl font-bold my-0'})[0].text
print(name)
location=soup.findAll('a',{'class':'Button_textLink__ONaN7 Button_grayColor__DiCOL Button_mdSize__prgpT','href':'#location'})[0].text
print(location)
country=location.split()[-1]
print(country)
trip_advisor=soup.findAll('a',{'class':'Button_textLink__ONaN7 Button_grayColor__DiCOL Button_mdSize__prgpT','rel':'noopener noreferrer'})
if len(trip_advisor)>0:
    trip_advisor=trip_advisor[0]['href']
else:
    trip_advisor=None
print(trip_advisor)
loc_description=soup.findAll('div',{'class':'CollapsibleContent_content__Kc4nd'})[0].text
print(loc_description[:200])

cleaned_images_links=[]
images_links=soup.findAll('img',{'class':'object-cover object-center'})
for link in images_links:
    if 'https' in link['src']:
        cleaned_images_links.append(link['src'])

print(cleaned_images_links)

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import asyncio
import aiohttp
import nest_asyncio

nest_asyncio.apply()


In [ ]:
centers_data = pd.DataFrame({'title': [0], 'location': [0], 'country': [0], 'trip_advisor': [0], 'description':[0],'url': [0]})

In [ ]:
centers_data.to_csv('centers_bookretreats.csv', encoding='utf-8',index=False)

In [ ]:
centers_data=pd.read_csv('centers_bookretreats.csv')

In [ ]:
cleaned_links=[]

pages_number=327
workers=50

pages_step=int(pages_number/workers)
diff=pages_number-pages_step*workers

pages_start=[pages_step*i+1 for i in range(workers)]
pages_end=[pages_step*i for i in range(1,workers+1)]
pages_end[-1]+=diff

async def get_links(start, end):
    global cleaned_links
    for i in range(start, end+1):
        print(i)
        url = f'https://bookretreats.com/centers/search?pageNumber={i}'

        async with aiohttp.ClientSession() as session:
            async with session.get(url) as response:

                response = await response.text()
                soup = BeautifulSoup(response, 'html.parser')

                links=soup.findAll('a', {'class': 'sm:col-span-2 flex flex-col justify-between flex-grow p-2'})
                for link in links:
                    cleaned_links.append('https://bookretreats.com'+link['href'])
                np.save('center_links.npy',cleaned_links)



futures = [get_links(pages_start[i],pages_end[i]) for i in range(workers)]

loop = asyncio.get_event_loop()
loop.run_until_complete(asyncio.wait(futures))
# loop.close()

In [ ]:
pages_number=5783
workers=100
folder='centres_images'
img_links=[]

pages_step=int(pages_number/workers)
diff=pages_number-pages_step*workers

pages_start=[pages_step*i+1 for i in range(workers)]
pages_end=[pages_step*i for i in range(1,workers+1)]
pages_end[-1]+=diff

cleaned_links=np.load('center_links.npy')


async def get_center(start, end):
    global centers_data
    global img_links
    global cleaned_links
    for i in range(start, end+1):

        center_url = cleaned_links[i]


        async with aiohttp.ClientSession() as center_session:
            async with center_session.get(center_url) as center_response:

                center_response =  await center_response.text()
                soup = BeautifulSoup(center_response, 'html.parser')

                print(f'iter {i}/{end+1}')
                # 1
                name=soup.findAll('h1',{'class':'text-2xl sm:text-3xl font-bold my-0'})[0].text
                # 2
                location=soup.findAll('a',{'class':'Button_textLink__ONaN7 Button_grayColor__DiCOL Button_mdSize__prgpT','href':'#location'})[0].text
                # 3
                country=location.split(',')[-1][1:]
                # 4
                trip_advisor=soup.findAll('a',{'class':'Button_textLink__ONaN7 Button_grayColor__DiCOL Button_mdSize__prgpT','rel':'noopener noreferrer'})
                if len(trip_advisor)>0:
                    trip_advisor=trip_advisor[0]['href']
                else:
                    trip_advisor=None
                # 5
                loc_description=soup.findAll('div',{'class':'CollapsibleContent_content__Kc4nd'})[0].text


                cleaned_images_links=[]
                images_links=soup.findAll('img',{'class':'object-cover object-center'})
                for link in images_links:
                    if 'https' in link['src']:
                        cleaned_images_links.append(link['src'])

                img_links.append((name,cleaned_images_links))
                np.save('img_links.npy', img_links)

                # for i,link in enumerate(cleaned_images_links):
                #
                #     image = requests.get(link).content
                #     with open(f'{folder}/{name}_{i}.jpg', 'wb') as handler:
                #         handler.write(image)

                # centers_data=pd.read_csv('centers_bookretreats.csv')

                centers_data = centers_data.append({'title': name, 'location': location, 'country': country, 'trip_advisor': trip_advisor, 'description': loc_description,'url': center_url}, ignore_index=True)
                centers_data.to_csv('centers_bookretreats.csv', encoding='utf-8',index=False)


futures = [get_center(pages_start[i],pages_end[i]) for i in range(workers)]

loop = asyncio.get_event_loop()
loop.run_until_complete(asyncio.wait(futures))
loop.close()

In [ ]:
centers_data.info()

In [ ]:
cleaned_df=centers_data.dropna(subset=['url'])

In [ ]:
cleaned_df.to_csv('centers_bookretreats.csv', encoding='utf-8',index=False)

In [ ]:
cleaned_df.info()

In [ ]:
cleaned_df[cleaned_df.title=='Ayur Yoga School']

In [ ]:
centers_data[centers_data.title=='Ayur Yoga School']

In [ ]:
centers_data[centers_data.url==None].info()

In [ ]:
contries=[]
for line in centers_data['location']:
    text=line.split(',')[-1]
    if text[0]==' ':
        text=text[1:]
    contries.append(text)

In [ ]:
values = contries
counts = Counter(values)
contries_x = []
contries_y = []
for k, count in counts.most_common():
    contries_x.append(k)
    contries_y.append(count)
    print(k)